# tensorflow 주요 아키텍처

+ 고수준 API : 케라스, 에스티메이터API
+ 저수준 API : Python wrapper (tf.nn)
+ C++ Tensorflow : c++ 계층


### 데이터 준비(MNIST)

In [1]:
import tensorflow as tf

num_classes =10 
img_rows, img_cols = 28,28
num_channels =1
input_shape = (img_rows, img_cols, num_channels)

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train/ 255.0, x_test/255.0

11493376/11490434 [==============================] - 0s 0us/step


tf.keras.datasets 모듈은 고전적인 데이터셋을 내려 받고 인스턴스화하기 위해 빠르게 접근할 수 있게 지원한다.

### 모델 구성

In [3]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten()) # (28x28) --> (1 x 784)
model.add(tf.keras.layers.Dense(128, activation = 'relu')) # (1 x 784) x (784 * 128) --> (1* 128)
model.add(tf.keras.layers.Dense(num_classes, activation = 'softmax')) #(1 * 128) x (128 * num_classes) --> (1 * numclasses)

### 모델 훈련

In [7]:
model.compile(optimizer = 'sgd', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, epochs = 5, verbose = 1, validation_data = (x_test, y_test)) 

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.6341 - accuracy: 0.8426 - val_loss: 0.3541 - val_accuracy: 0.9028
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3378 - accuracy: 0.9053 - val_loss: 0.2939 - val_accuracy: 0.9163
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2901 - accuracy: 0.9181 - val_loss: 0.2600 - val_accuracy: 0.9281
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2603 - accuracy: 0.9263 - val_loss: 0.2385 - val_accuracy: 0.9315
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2370 - accuracy: 0.9340 - val_loss: 0.2211 - val_accuracy: 0.9384


### 모델 구성 보기

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


# Tensorflow 2와 케라스

+ tensor: N차원 배열로 생각할 수 있음
+ 고정값: tf.constant
+ 변경되는 값: tf.Variable
----------------------------------------------------------------
- 타입: string, float32, float16, int8
- 형상: ()이면 스칼라고, (n)이면 크기가 n인 벡터, (n,m)이면 크기가 nxm인 2차원 행렬
- 순위: 차원 개수, 0이면 스칼라, 1이면 벡터, 2이면 2차원 행렬이다. 

### 텐서플로 그래프
입력과 출력으로 텐서를 사용, 그래프를 사용해 연산을 표현한다.
+ 장점
  > CPU에서 일부 연산을 실행하고 GPU에서 남은 연산을 실행한다.

  > 분산 모델의 경우 그래프의 다양한 부분을 여러 다른 컴퓨터에서 실행한다.
  
  > 불필요한 연산을 피하기 위해 그래프를 최적화해 계산 성능을 개선한다.


In [9]:
a  = tf.constant([1,2,3])
b = tf.constant([0,0,1])
c = tf.add(a,b)
print(c)

tf.Tensor([1 2 4], shape=(3,), dtype=int32)


### 텐서플로2 그래프 생성

In [10]:
def compute(a,b,c):
  d = a*b+c
  e = a*b*c
  return d,e

tensorflow2에서는 모든 연산이 정의되는 시점에 실행됨

### 텐서플로 오토프래프와 tf.function

In [12]:
@tf.function
def compute(a,b,c):
  d = a*b+c
  e = a*b*c
  return d,e

@tf.function을 함으로써 tensorflow2에서 그래프를 투명하게 생성한다. 
--> 그래프를 최적화한다.

#### 오토그래프를 사용할때
+ 모델을 다른 기기로 내보내야 할 때
+ 성능이 무엇보다 중요하고 그래프 최적화를 통해 속도 개선이 가능할 때

### 자동 미분
+ 그래프의 장점으로 자동 미분이 가능하다. --> "Gradient tape"
